In [ ]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 412.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=a4a0950023a1e214c9fdf8438c28bdeab0f805e7b6a7d4717dca9aeace1d81f8
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=b59a8f4379fd819b9fab84f763eff8a07936d5509d95cceae989f2749e3f3ba0
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import time
import requests
from torchvision.datasets import ImageFolder
from torchvision import models
from PIL import Image
from shutil import copyfile
import os

# os.chdir('/content/drive/My Drive')
from datasets.cityscapes import CityscapesDataset
from datasets.gta5 import GTA5

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cpu'

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((1280,720)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_lbl_transform = transforms.Compose([
    transforms.Resize((1280,720), interpolation=Image.Resampling.NEAREST),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((1024,512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_lbl_transform = transforms.Compose([
    transforms.Resize((1024,512), interpolation=Image.Resampling.NEAREST),
    transforms.ToTensor(),
])

In [7]:
train_dataset = GTA5('datasets\\GTA5', transform=[train_transform,train_lbl_transform]) # /content/drive/MyDrive/datasets/GTA5
val_dataset = CityscapesDataset('datasets\\Cityscapes\\Cityspaces', transform=[val_transform,val_lbl_transform]) # /content/drive/MyDrive/datasets/Cityscapes/Cityscapes/Cityspaces

train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True) #16
val_dataloader = DataLoader(val_dataset, batch_size=6, shuffle=True) #16

In [13]:
for images, labels in train_dataloader:
    # Access the first image and label
    image, label = images[0], labels[0]
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)
    print("Label:", label)
    print(torch.argmax(labels, dim=1).shape)
    print(labels.shape)    

    break

Image shape: torch.Size([3, 1280, 720])
Label shape: torch.Size([1, 1280, 720])
Label: tensor([[[0.0392, 0.0392, 0.0392,  ..., 0.0392, 0.0392, 0.0392],
         [0.0392, 0.0392, 0.0392,  ..., 0.0392, 0.0392, 0.0392],
         [0.0392, 0.0392, 0.0392,  ..., 0.0392, 0.0392, 0.0392],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])
torch.Size([6, 1280, 720])
torch.Size([6, 1, 1280, 720])
torch.Size([6, 3, 1280, 720])


In [11]:
for images, labels in val_dataloader:
    # Access the first image and label
    image, label = images[0], labels[0]
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)
    print("Label:", label)
    print(torch.argmax(labels, dim=1).shape)
    print(labels.shape)

    break

Image shape: torch.Size([3, 1024, 512])
Label shape: torch.Size([1, 1024, 512])
Label: tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])
torch.Size([6, 1024, 512])
torch.Size([6, 1, 1024, 512])


In [14]:
# Load pre-trained model

from models.bisenet.build_bisenet import BiSeNet
bisenet = BiSeNet(num_classes = 19, context_path='resnet18').to(device)

In [ ]:
bisenet

BiSeNet(
  (saptial_path): Spatial_path(
    (convblock1): ConvBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (convblock2): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (convblock3): ConvBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (context_path): resnet18(
    (features): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [15]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [16]:
# Get the length of class_names (one output unit for each class)
output_shape = 19 # 0 to 18

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(bisenet.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [22]:
def train(model, optimizer, dataloader, loss_fn, alpha = 1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(dataloader):
#         inputs, targets = inputs.cuda(), targets.cuda()
        targets = torch.argmax(targets, dim=1)  

        outputs, cx1_sup, cx2_sup = model(inputs)
        
        # main loss
        main_loss = loss_fn(outputs, targets)

        # supervision losses
        sup1_loss = loss_fn(cx1_sup, targets)
        sup2_loss = loss_fn(cx2_sup, targets)

        # combined loss
        loss = main_loss + alpha * (sup1_loss + sup2_loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        
        
    train_loss = running_loss / len(dataloader)


def test(model, dataloader, loss_fn):

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
#             inputs, targets = inputs.cuda(), targets.cuda()
            targets = torch.argmax(targets, dim=1)

            outputs = model(inputs)

            loss = loss_fn(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
                 
            
    test_loss = test_loss / len(dataloader)

    return test_loss

In [23]:
from utils import poly_lr_scheduler

In [24]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
epochs = 1 #50

for ep in range(epochs):

    train(bisenet, optimizer, train_dataloader, loss_fn)
    test_loss = test(bisenet, val_dataloader, loss_fn)

    ## Inserting learning rate decay
    curr_lr = poly_lr_scheduler(optimizer = optimizer, init_lr = 2.5e-2, iter = ep, lr_decay_iter=1, max_iter=epochs, power=0.9)

    print(f"Current learning rate: {curr_lr}")
    print(f"Test loss: {test_loss}")
    print("\n")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

Current learning rate: 0.025
Test loss: 0.01509451684150987


[INFO] Total training time: 87.822 seconds


In [ ]:
# latency and FPS

import time
from torch.utils.data import RandomSampler

iterations = 1000
random_sampler = RandomSampler(val_dataset, replacement=False, num_samples=iterations)
random_dataloader = DataLoader(val_dataset, batch_size=1, sampler=random_sampler)

latency = []
fps = []
with torch.no_grad():
    for inputs, _ in random_dataloader:
        inputs = inputs.cuda()
        start = time.time()
        prediction = bisenet(inputs)
        end = time.time()
        latency_i = end - start
        latency.append(latency_i)
        fps_i = 1/latency_i
        fps.append(fps_i)

meanLatency = np.mean(latency)*1000
stdLatency = np.std(latency)*1000
meanfps = np.mean(fps)
stdfps = np.std(fps)

print("Mean Latency:", meanLatency)
print("Std Latency:", stdLatency)
print("Mean FPS:", meanfps)
print("Std FPS:", stdfps)

Mean Latency: 5.290594577789307
Std Latency: 4.615041586624911
Mean FPS: 204.40439728350637
Std FPS: 35.12336617192247


In [ ]:
# FLOPs and parameters

from fvcore.nn import FlopCountAnalysis, flop_count_table

height = 512
width = 1024
image = torch.zeros((3, height, width))

flops = FlopCountAnalysis(bisenet, image)
print(flop_count_table(flops))

In [25]:
from utils import fast_hist
from utils import per_class_iou

def mean_iou(outputs, labels, num_classes):
    hist = 0
    for i in range(len(outputs)):
        output, label = outputs[i].cpu().numpy().reshape(-1,), labels[i].cpu().numpy().reshape(-1,)
        hist += fast_hist(label, output, num_classes)
    class_iou = per_class_iou(hist)
    return np.mean(class_iou), class_iou

In [31]:
def getting_data_results_analysis(model, dataloader):

    model.eval()
    samples = []
    all_targets = []
    preds = []
    probs = []
    total_miou = 0.0
    total_per_class_iou = np.zeros(19)
    n_batches = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):

#             inputs, targets = inputs.cuda(), targets.cuda()
            targets = torch.argmax(targets, dim=1)
            outputs = model(inputs)

            softmax = torch.nn.Softmax(dim=1)
            prob = softmax(outputs)
            predicted = torch.argmax(prob, dim=1)
            probs.append(prob.max().item())
            preds.append(predicted)
            all_targets.append(targets)
            samples.append(inputs)

            outputs = torch.argmax(outputs, dim=1)
            miou, class_iou = mean_iou(outputs, targets, 19)
            total_miou += miou
            total_per_class_iou += class_iou
            n_batches += 1
            

    samples = torch.cat(samples, dim=0)
    preds = torch.cat(preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    probs = torch.cat(probs, dim=0)

    avg_miou = total_miou / n_batches
    avg_miou_per_class = total_per_class_iou / n_batches

    return samples, preds, all_targets, probs, avg_miou, avg_miou_per_class

In [32]:
# first test executed on cpu just to see if the code was working well (inserted a break after the very first batchs)

samples, preds, target, probs, avg_miou, avg_miou_per_class = getting_data_results_analysis(bisenet, val_dataloader)

print(f"Average mIoU:{avg_miou}")
print(f"Average mIoU per class:{avg_miou_per_class}")

Average mIoU:6.20222928229251e-05
Average mIoU per class:[0.00117842 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
